# Setup

In [1]:
!pip install -qU accelerate diffusers gradio torch safetensors transformers
!pip install -q git+https://github.com/deepseek-ai/Janus

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 342.1/342.1 kB 29.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 MB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.0/322.0 kB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 766.7/766.7 MB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 53.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 32.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Bibliotek:
- accelerate: służy do przyspieszenia obliczeń na GPU
- diffusers: zawiera implementacje modeli generatywnych
- gradio: pozwala tworzyć interfejsy graficzne dla modeli AI
- torch: główna biblioteka do uczenia maszynowego PyTorch
- safetensors: zapewnia bezpieczne przechowywanie modeli
- Janus: specjalistyczna biblioteka do zaawansowanych zadań związanych z przetwarzaniem języka naturalnego

In [ ]:
import gradio as gr
import torch
from transformers import AutoConfig, AutoModelForCausalLM
from janus.models import VLChatProcessor
from PIL import Image

import numpy as np
import os

# enables faster downloads from hugging face
os.environ["HF_HUB_ENABLE_HF_TRANSFER"] = "1"

Python version is above 3.10, patching the collections module.


/usr/local/lib/python3.11/dist-packages/transformers/models/auto/image_processing_auto.py:594: FutureWarning: The image_processor_class argument is deprecated and will be removed in v4.42. Please use `slow_image_processor_class`, or `fast_image_processor_class` instead
  warnings.warn(


Z pakietu transformers pobierane są dwa kluczowe komponenty: `AutoConfig` i `AutoModelForCausalLM`. AutoConfig pozwala na automatyczne wczytanie konfiguracji modelu, a AutoModelForCausalLM to klasa bazowa dla modeli generatywnych, które przewidują następne elementy sekwencji (np. kolejne słowa w tekście).

Z biblioteki janus importowane są specjalistyczne komponenty: `VLChatProcessor`, który przetwarza dane wejściowe do formatu zrozumiałego dla modelu.

In [3]:
class CFG:
  model = "deepseek-ai/Janus-Pro-1B"
  device = 'cuda'
  dtype = torch.bfloat16

In [4]:
config = AutoConfig.from_pretrained(CFG.model)
language_config = config.language_config
language_config._attn_implementation = 'eager'
vl_gpt = AutoModelForCausalLM.from_pretrained(CFG.model,
                                             language_config=language_config,
                                             trust_remote_code=True)
vl_gpt = vl_gpt.to(CFG.dtype).cuda()

vl_chat_processor = VLChatProcessor.from_pretrained(CFG.model)
tokenizer = vl_chat_processor.tokenizer
cuda_device = 'cuda'


config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/4.18G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/4.18G [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/346 [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.48, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


...

tokenizer.json:   0%|          | 0.00/4.72M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/344 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama_fast.LlamaTokenizerFast'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565 - if you loaded a llama tokenizer from a GGUF file you can ignore this message.


processor_config.json:   0%|          | 0.00/210 [00:00<?, ?B/s]

Some kwargs in processor config are unused and will not have any effect: mask_prompt, ignore_id, num_image_tokens, add_special_token, sft_format, image_tag. 


- Następnie kod wyodrębnia konfigurację dotyczącą części językowej modelu (language_config) i wprowadza w niej ważną zmianę: **ustawia implementację mechanizmu uwagi (_attn_implementation) na tryb "eager"**. Mechanizm uwagi to kluczowy element modeli językowych, który pozwala im "skupiać się" na istotnych częściach wejścia. Tryb `eager` oznacza natychmiastowe wykonywanie obliczeń, co jest przydatne podczas debugowania i w niektórych specyficznych zastosowaniach.

- Kolejny krok to wczytanie samego modelu przy użyciu AutoModelForCausalLM.from_pretrained(). Parametr trust_remote_code=True pozwala na wykonanie kodu pobranego wraz z modelem, co jest konieczne dla specjalistycznych modeli jak Janus. Model jest następnie przenoszony na GPU (cuda()) i konwertowany do określonego formatu liczb (CFG.dtype).

- Kod tworzy również procesor VLChatProcessor, który jest odpowiedzialny za przygotowanie danych wejściowych (tekstu i obrazów) do formatu zrozumiałego dla modelu. Z tego procesora wyodrębniony zostaje tokenizer - narzędzie, które zamienia tekst na liczby, którymi może operować model.

# Functions

Adapted from https://huggingface.co/spaces/afrideva/Janus-Pro-1b


In [5]:
def multimodal_understanding(image, question, seed, top_p, temperature):
    # Clear CUDA cache before generating
    torch.cuda.empty_cache()

    # set seed
    torch.manual_seed(seed)
    np.random.seed(seed)
    torch.cuda.manual_seed(seed)

    conversation = [
        {
            "role": "<|User|>",
            "content": f"<image_placeholder>\n{question}",
            "images": [image],
        },
        {"role": "<|Assistant|>", "content": ""},
    ]

    pil_images = [Image.fromarray(image)]
    prepare_inputs = vl_chat_processor(
        conversations=conversation, images=pil_images, force_batchify=True).to(cuda_device, dtype = CFG.dtype)


    inputs_embeds = vl_gpt.prepare_inputs_embeds(**prepare_inputs)

    outputs = vl_gpt.language_model.generate(
        inputs_embeds=inputs_embeds,
        attention_mask=prepare_inputs.attention_mask,
        pad_token_id=tokenizer.eos_token_id, bos_token_id=tokenizer.bos_token_id,
        eos_token_id=tokenizer.eos_token_id,
        max_new_tokens=512,
        do_sample=False if temperature == 0 else True, use_cache=True,
        temperature=temperature, top_p=top_p,
    )

    answer = tokenizer.decode(outputs[0].cpu().tolist(), skip_special_tokens=True)
    return answer

Ta funkcja stanowi serce systemu - jest odpowiedzialna za zrozumienie obrazu i odpowiadanie na pytania o nim.

Na początku funkcja czyści pamięć GPU za pomocą torch.cuda.empty_cache(). Jest to ważne działanie zapobiegawcze, które usuwa niepotrzebne dane z poprzednich operacji i zapewnia maksimum dostępnej pamięci dla nowego zadania.

Następnie ustawiane są ziarna (seeds) dla generatorów liczb losowych w różnych komponentach systemu. Ustawienie tych samych ziaren zapewnia powtarzalność wyników - przy tych samych danych wejściowych i ziarnie otrzymamy te same odpowiedzi. Jest to niezbędne do debugowania i testowania systemu.

Kolejnym krokiem jest utworzenie struktury konwersacji. Ma ona formę listy słowników, gdzie każdy słownik reprezentuje jedną wypowiedź. Pierwsza wypowiedź pochodzi od użytkownika i zawiera zarówno pytanie jak i obraz (znacznik <image_placeholder> wskazuje miejsce, gdzie należy wstawić obraz). Druga wypowiedź to puste miejsce na odpowiedź asystenta.

Obraz przekazany do funkcji jest konwertowany na format PIL (Python Imaging Library). Jest to standardowy format używany w Pythonie do pracy z obrazami, który pozwala na ich łatwe przetwarzanie i manipulację.

Następnie vl_chat_processor przygotowuje dane wejściowe, łącząc tekst i obraz w format zrozumiały dla modelu. Parametr force_batchify=True wymusza przetwarzanie danych jako wsadu, nawet jeśli mamy tylko jeden przykład. Przygotowane dane są przenoszone na GPU i konwertowane na odpowiedni format liczbowy.

W kolejnym kroku model przetwarza te dane na osadzone reprezentacje (embeddings), które są następnie używane do generowania odpowiedzi. Proces generowania jest kontrolowany przez szereg parametrów:
- max_new_tokens określa maksymalną długość generowanej odpowiedzi
- do_sample włącza lub wyłącza losowość w generowaniu (zależy od temperatury)
- temperature kontroluje "kreatywność" modelu - wyższe wartości dają bardziej zróżnicowane odpowiedzi
- top_p określa, jaka część rozkładu prawdopodobieństwa jest brana pod uwagę przy wyborze następnego tokenu

Na końcu wygenerowana sekwencja tokenów jest dekodowana z powrotem na tekst, pomijając specjalne tokeny techniczne. Ten tekst stanowi odpowiedź modelu na pytanie o obrazie.


In [6]:
def generate(input_ids, width, height,
             temperature: float = 1,
             parallel_size: int = 5,
             cfg_weight: float = 5,
             image_token_num_per_image: int = 576,
             patch_size: int = 16):
    # Clear CUDA cache before generating
    torch.cuda.empty_cache()

    tokens = torch.zeros((parallel_size * 2, len(input_ids)), dtype=torch.int).to(cuda_device)
    for i in range(parallel_size * 2):
        tokens[i, :] = input_ids
        if i % 2 != 0:
            tokens[i, 1:-1] = vl_chat_processor.pad_id
    inputs_embeds = vl_gpt.language_model.get_input_embeddings()(tokens)
    generated_tokens = torch.zeros((parallel_size, image_token_num_per_image), dtype=torch.int).to(cuda_device)

    pkv = None
    for i in range(image_token_num_per_image):
        with torch.no_grad():
            outputs = vl_gpt.language_model.model(inputs_embeds=inputs_embeds,
                                                use_cache=True,
                                                past_key_values=pkv)
            pkv = outputs.past_key_values
            hidden_states = outputs.last_hidden_state
            logits = vl_gpt.gen_head(hidden_states[:, -1, :])
            logit_cond = logits[0::2, :]
            logit_uncond = logits[1::2, :]
            logits = logit_uncond + cfg_weight * (logit_cond - logit_uncond)
            probs = torch.softmax(logits / temperature, dim=-1)
            next_token = torch.multinomial(probs, num_samples=1)
            generated_tokens[:, i] = next_token.squeeze(dim=-1)
            next_token = torch.cat([next_token.unsqueeze(dim=1), next_token.unsqueeze(dim=1)], dim=1).view(-1)

            img_embeds = vl_gpt.prepare_gen_img_embeds(next_token)
            inputs_embeds = img_embeds.unsqueeze(dim=1)



    patches = vl_gpt.gen_vision_model.decode_code(generated_tokens.to(dtype=torch.int),
                                                 shape=[parallel_size, 8, width // patch_size, height // patch_size])

    return generated_tokens.to(dtype=torch.int), patches


Ta funkcja jest odpowiedzialna za generowanie obrazów i stanowi przykład zaawansowanego zastosowania techniki Classifier-Free Guidance (CFG) w modelach generatywnych. Przeanalizujmy jej działanie krok po kroku.

Funkcja przyjmuje kilka kluczowych parametrów: identyfikatory tokenów wejściowych (input_ids), wymiary generowanego obrazu (width, height), temperaturę wpływającą na losowość generacji, liczbę równoległych generacji (parallel_size), wagę CFG oraz parametry techniczne dotyczące tokenizacji obrazu.

Na początku, podobnie jak w poprzedniej funkcji, czyszczona jest pamięć GPU. Następnie tworzona jest macierz tokenów o podwójnym rozmiarze równoległym - dla każdej generacji tworzone są dwie ścieżki: warunkowa i bezwarunkowa. Jest to kluczowy element techniki CFG.

W głównej pętli generacji dla każdego tokenu obrazu wykonywane są następujące operacje:

Model przetwarza osadzone wejścia (inputs_embeds), wykorzystując mechanizm cache'owania kluczy i wartości (past_key_values) dla zwiększenia wydajności. To znacząco przyspiesza generację, ponieważ nie musimy ponownie przetwarzać wcześniejszych tokenów.

Następnie model oblicza logity (nieznormalizowane prawdopodobieństwa) dla następnego tokenu. Tu następuje najważniejszy element CFG - logity są obliczane osobno dla ścieżki warunkowej i bezwarunkowej, a następnie łączone według wzoru:
logits = logit_uncond + cfg_weight * (logit_cond - logit_uncond)

Ten wzór pozwala kontrolować, jak mocno model trzyma się warunków początkowych. Większa wartość cfg_weight sprawia, że generacja jest bardziej "posłuszna" względem zadanych warunków.

Prawdopodobieństwa są obliczane poprzez zastosowanie funkcji softmax z zadaną temperaturą. Wyższe wartości temperatury prowadzą do bardziej losowych generacji, podczas gdy niższe wartości sprawiają, że model częściej wybiera najbardziej prawdopodobne tokeny.

Wygenerowane tokeny są następnie dekodowane przez model wizyjny do postaci "łatek" (patches) obrazu. Jest to proces odwrotny do tokenizacji obrazu - model składa małe fragmenty w pełny obraz.

Cały ten proces można porównać do pracy artysty, który tworzy obraz element po elemencie, ale jednocześnie utrzymuje dwie wizje w głowie: jedną ściśle związaną z zamierzonym efektem i drugą bardziej swobodną. Następnie łączy te dwie wizje w proporcjach, które pozwalają zachować równowagę między kreatywnością a zgodnością z założeniami.

Ta implementacja pokazuje, jak zaawansowane techniki generacji AI pozwalają kontrolować proces twórczy, balansując między swobodą a precyzją generacji.

In [7]:
def unpack(dec, width, height, parallel_size=5):
    dec = dec.to(torch.float32).cpu().numpy().transpose(0, 2, 3, 1)
    dec = np.clip((dec + 1) / 2 * 255, 0, 255)

    visual_img = np.zeros((parallel_size, width, height, 3), dtype=np.uint8)
    visual_img[:, :, :] = dec

    return visual_img

Ta funkcja służy do przekształcania wewnętrznej reprezentacji obrazu stworzonej przez model na faktyczny obraz cyfrowy, który można wyświetlić. Jest to ostatni krok w procesie generacji obrazu, można go porównać do procesu wywoływania zdjęcia w ciemni fotograficznej.

Pierwsza linia kodu przenosi dane z GPU na CPU i zmienia ich format na 32-bitowe liczby zmiennoprzecinkowe. Następnie wykonywana jest transpozycja macierzy, która zmienia kolejność wymiarów. Jest to konieczne, ponieważ model pracuje na danych w formacie, który jest optymalny dla obliczeń (BCHW - batch, channels, height, width), ale do wyświetlenia potrzebujemy formatu, który odpowiada rzeczywistej strukturze obrazu (BHWC - batch, height, width, channels).

Kolejny krok to normalizacja wartości pikseli. Wewnętrznie model pracuje na wartościach w zakresie od -1 do 1, które musimy przekształcić na standardowy zakres wartości pikseli od 0 do 255. Wzór (dec + 1) / 2 * 255 najpierw przesuwa zakres wartości na 0-2, potem skaluje go do 0-1, a na końcu do 0-255. Funkcja np.clip gwarantuje, że żadna wartość nie wykroczy poza ten zakres.

Na końcu tworzona jest nowa tablica o wymiarach odpowiadających liczbie równoległych generacji (parallel_size), szerokości, wysokości i trzem kanałom kolorów (RGB). Typ danych uint8 oznacza, że każdy piksel będzie przechowywany jako 8-bitowa liczba całkowita bez znaku, co jest standardowym formatem dla obrazów cyfrowych.

Ta funkcja jest niezbędnym mostem między światem matematycznych reprezentacji używanych przez model AI a światem rzeczywistych obrazów cyfrowych, które możemy zobaczyć na ekranie. Bez tej transformacji wygenerowane przez model dane pozostałyby niezrozumiałą macierzą liczb.

In [8]:
def generate_image(prompt,
                   seed=None,
                   guidance=5,
                   t2i_temperature=1.0):
    # Clear CUDA cache and avoid tracking gradients
    torch.cuda.empty_cache()
    # Set the seed for reproducible results
    if seed is not None:
        torch.manual_seed(seed)
        torch.cuda.manual_seed(seed)
        np.random.seed(seed)
    width = 384
    height = 384
    parallel_size = 5

    with torch.no_grad():
        messages = [{'role': '<|User|>', 'content': prompt},
                    {'role': '<|Assistant|>', 'content': ''}]
        text = vl_chat_processor.apply_sft_template_for_multi_turn_prompts(conversations=messages,
                                                                   sft_format=vl_chat_processor.sft_format,
                                                                   system_prompt='')
        text = text + vl_chat_processor.image_start_tag

        input_ids = torch.LongTensor(tokenizer.encode(text))
        output, patches = generate(input_ids,
                                   width // 16 * 16,
                                   height // 16 * 16,
                                   cfg_weight=guidance,
                                   parallel_size=parallel_size,
                                   temperature=t2i_temperature)
        images = unpack(patches,
                        width // 16 * 16,
                        height // 16 * 16,
                        parallel_size=parallel_size)

        return [Image.fromarray(images[i]).resize((768, 768), Image.LANCZOS) for i in range(parallel_size)]


Ta funkcja jest głównym punktem wejścia do generowania obrazów na podstawie opisu tekstowego. Przeanalizujmy, jak przekształca ona prosty tekst w złożoną wizualizację.

Na początku, jak w każdej funkcji pracującej z GPU, czyścimy pamięć karty graficznej. Jest to istotne, ponieważ generowanie obrazów wymaga znacznej ilości pamięci, a pozostałości z poprzednich operacji mogłyby prowadzić do jej niedoboru.

Funkcja przyjmuje kilka kluczowych parametrów sterujących procesem generacji. Ziarno losowości (seed) pozwala na odtwarzalne generacje - ten sam tekst z tym samym ziarnem zawsze wygeneruje identyczne obrazy. Jest to niezwykle przydatne podczas testowania i debugowania. Parametr guidance kontroluje, jak ściśle model trzyma się opisu tekstowego, a t2i_temperature wpływa na kreatywność generacji.

Wymiary generowanego obrazu są ustawione na 384x384 piksele. Warto zauważyć, że są one następnie dostosowywane do wielokrotności 16 (szerokość // 16 * 16). Ta operacja jest konieczna, ponieważ model działa na "łatkach" (patches) o rozmiarze 16x16 pikseli. Bez tego dostosowania moglibyśmy otrzymać niepełne lub zniekształcone łatki na brzegach obrazu.

Właściwy proces generacji rozpoczyna się od przygotowania kontekstu konwersacji. Model jest skonstruowany tak, by działać w formacie dialogowym - prompt jest formatowany jako wypowiedź użytkownika, po której następuje pusta odpowiedź asystenta. To pokazuje, jak model łączy cechy klasycznego chatbota z możliwościami generacji obrazów.

Istotne jest dodanie image_start_tag - specjalnego znacznika informującego model, że od tego miejsca powinien generować obraz, a nie tekst. Jest to swego rodzaju przełącznik trybu pracy modelu.

Podczas generacji model tworzy równolegle pięć różnych wersji obrazu (parallel_size = 5). Jest to efektywne podejście, ponieważ współczesne GPU świetnie radzą sobie z równoległym przetwarzaniem, a użytkownik otrzymuje więcej opcji do wyboru.

Na końcu generowane obrazy są powiększane do rozmiaru 768x768 pikseli przy użyciu algorytmu Lanczos. Jest to zaawansowana metoda skalowania, która zachowuje ostrość krawędzi i detale lepiej niż prostsze metody jak interpolacja liniowa.


# The app

In [9]:
with gr.Blocks() as demo:
    gr.Markdown(value="# Multimodal Understanding")
    with gr.Row():
        image_input = gr.Image()
        with gr.Column():
            question_input = gr.Textbox(label="Question")
            und_seed_input = gr.Number(label="Seed", precision=0, value=42)
            top_p = gr.Slider(minimum=0, maximum=1, value=0.95, step=0.05, label="top_p")
            temperature = gr.Slider(minimum=0, maximum=1, value=0.1, step=0.05, label="temperature")

    understanding_button = gr.Button("Chat")
    understanding_output = gr.Textbox(label="Response")

    gr.Markdown(value="# Text-to-Image Generation")



    with gr.Row():
        cfg_weight_input = gr.Slider(minimum=1, maximum=10, value=5, step=0.5, label="CFG Weight")
        t2i_temperature = gr.Slider(minimum=0, maximum=1, value=1.0, step=0.05, label="temperature")

    prompt_input = gr.Textbox(label="Prompt")
    seed_input = gr.Number(label="Seed (Optional)", precision=0, value=12345)

    generation_button = gr.Button("Generate Images")

    image_output = gr.Gallery(label="Generated Images", columns=2, rows=2, height=300)


    understanding_button.click(
        multimodal_understanding,
        inputs=[image_input, question_input, und_seed_input, top_p, temperature],
        outputs=understanding_output
    )

    generation_button.click(
        fn=generate_image,
        inputs=[prompt_input, seed_input, cfg_weight_input, t2i_temperature],
        outputs=image_output
    )

demo.launch(debug = True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://ec7515c89fc8ca9200.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Ten fragment kodu tworzy interaktywny interfejs graficzny używając biblioteki Gradio. Interfejs jest podzielony na dwie główne sekcje: jedną do analizy obrazów i drugą do ich generowania. Przyjrzyjmy się, jak zbudowana jest każda z tych części.

W pierwszej sekcji, zatytułowanej "Multimodal Understanding", użytkownik może prowadzić rozmowę z modelem na temat obrazów. Interfejs zawiera pole do wczytania obrazu (image_input) oraz kolumnę kontrolek do interakcji:
- pole tekstowe na pytanie o obraz
- pole numeryczne do ustawienia ziarna losowości (domyślnie 42)
- suwak top_p kontrolujący różnorodność odpowiedzi (0-1, domyślnie 0.95)
- suwak temperatury wpływający na kreatywność modelu (0-1, domyślnie 0.1)
- przycisk "Chat" do rozpoczęcia rozmowy
- pole tekstowe wyświetlające odpowiedź modelu

Druga sekcja, "Text-to-Image Generation", pozwala na tworzenie obrazów z opisów tekstowych. Zawiera następujące elementy:
- suwak wagi CFG (1-10, domyślnie 5) kontrolujący, jak ściśle model trzyma się opisu
- suwak temperatury dla generacji obrazu (0-1, domyślnie 1.0)
- pole tekstowe na opis obrazu (prompt)
- pole numeryczne na opcjonalne ziarno losowości (domyślnie 12345)
- przycisk "Generate Images"
- galerię wygenerowanych obrazów (2x2 obrazy)

Najciekawszym elementem technicznym są dwa połączenia (click events) definiujące, co się dzieje po naciśnięciu przycisków:

Dla przycisku "Chat", funkcja multimodal_understanding otrzymuje pięć parametrów wejściowych: obraz, pytanie, ziarno, top_p i temperaturę. Wynik jest wyświetlany w polu odpowiedzi.

Dla przycisku "Generate Images", funkcja generate_image przyjmuje cztery parametry: opis, ziarno, wagę CFG i temperaturę. Wygenerowane obrazy są pokazywane w galerii.

Na końcu interfejs jest uruchamiany z opcją share=True, co oznacza, że będzie dostępny publicznie przez internet, nie tylko lokalnie na komputerze.
